Python 3.8

Project Instructions

The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called ds_jobs_transformed that stores the data in customer_train.csv much more efficiently. Specifically, they have set the following requirements:

    Columns containing categories with only two factors must be stored as Booleans (bool).
    Columns containing integers only must be stored as 32-bit integers (int32).
    Columns containing floats must be stored as 16-bit floats (float16).
    Columns containing nominal categorical data must be stored as the category data type.
    Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
    The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

If you call .info() or .memory_usage() methods on ds_jobs and ds_jobs_transformed after you've preprocessed it, you should notice a substantial decrease in memory usage.

How to approach the project

1. Exploratory data analysis

Load customer_train.csv to begin exploring the data to understand the contents and data types of the values in each column.
How to find the data types and contents of a column

    You can check the column names and assigned data types by calling .info() on the DataFrame.
    The .value_counts() method can be used to view the unique values and counts present in a column.

How to determine what a columns data type should be

    Columns containing only two unique values with yes/no style values (two-factor categories), should be converted to the bool data type.
    Columns containing a small number of unique values with no natural ordering should be set to the category data type, as they contain nominal categorical data.
    Columns containing categorical data with a natural ordering should be converted to ordered categories, as they contain ordinal categorical data.

2. Converting integers, floats, and unordered categories

Convert columns containing integers to the int32 type, floats to the float16 type, nominal categories to the category type, and two-factor categories to the bool type.
Converting data types

    The .astype() method can be called on a DataFrame column to convert its data type.
    For converting the two-factor categories, there are a few methods to approach this, but one of the quickest is to create a dictionary to map the existing values to the desired ones.
    To map values in a Series to new values, call the .map() method on the Series, passing it a dictionary with the following structure:


{
    'current_value1': 'new_value1',
    'current_value2': 'new_value2'
}


3. Converting ordered categories

Convert columns containing ordinal categorical data into ordered categories.
Creating ordered categorical data types

    To create an ordered categorical data type, use the pd.CategoricalDtype() function, which takes two arguments: an ordered list of categories (as strings), and ordered, which takes a Boolean indicating if the categories should be ordered.

    For each ordinal column, you'll need to create its own ordered categorical data type, with categories ordered to reflect their natural ordering (from lowest to highest). For example, to create an ordered category for a column containing Olympic medal results, you could use:


    medals = pd.CategoricalDtype(['Bronze', 'Silver', 'Gold'], ordered=True)

Creating an ordered list of work experience values

    In ascending order, the work experience values should be <1, then the numbers one through to twenty, then finish with >20.
    There are lots of ways to create this list:
        You could use the .unique() and .sort() methods on the column, then convert to a list and rearrange to ensure <1 and >20 are in the correct position.
        Create a list of numbers 1-20, map the numbers to strings, and concatenate ["<1"] and [">20"] to the beginning and end, respectively.

Converting columns into ordered categorical columns

    For each ordered categorical column, you'll need to convert its data type using .astype() and the data type object created with pd.CategoricalDtype().

4. Filtering on ordered categorical columns

Filter the DataFrame to only contain students with 10 or more years of experience at companies with at least 1000 employees.
Filtering DataFrames on ordered categorical columns

    Ordered categorical columns and numerical columns can be filtered using comparison operators such as >.
    You can combine multiple filters with an & operator.

![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [4]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!
# Import necessary libraries
import pandas as pd

# Load the dataset and create a copy
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs_transformed = ds_jobs.copy()

# EDA to help identify ordinal, nominal, and two-factor categories
for col in ds_jobs.select_dtypes("object").columns:
    print(ds_jobs_transformed[col].value_counts(), '\n')

# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

# Create a mapping dictionary of columns containing two-factor categories to convert to Booleans
two_factor_cats = {
    'relevant_experience': {'No relevant experience': False, 'Has relevant experience': True},
    'job_change': {0.0: False, 1.0: True}
}

# Loop through DataFrame columns to efficiently change data types
for col in ds_jobs_transformed:
    
    # Convert two-factor categories to bool
    if col in ['relevant_experience', 'job_change']:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].map(two_factor_cats[col])
    
    # Convert integer columns to int32
    elif col in ['student_id', 'training_hours']:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
    
    # Convert float columns to float16
    elif col == 'city_development_index':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')
    
    # Convert columns containing ordered categorical data to ordered categories using dict
    elif col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype(category)
    
    # Convert remaining columns to standard categories
    else:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')
        
# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]

# Congratulations, you completed the project!

city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64 

Male      13221
Female     1238
Other       191
Name: gender, dtype: int64 

Has relevant experience    13792
No relevant experience      5366
Name: relevant_experience, dtype: int64 

no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64 

Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64 

STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64 

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
1